#### This jupyter notebook scrapes emails in an email address and filter out emails based on the subject.

In [ ]:
#import the necessary modules, and then specify the credentials of your email account.
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import csv

# account credentials
username = "youremailaddress@provider.com"
password = "yourpassword"

In [ ]:
#connect to the IMAP server
# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL("imap.gmail.com")

# authenticate
imap.login(username, password)

Since I'm testing this on a gmail account, I've used [imap.gmail.com](imap.gmail.com) server, this [link](https://www.systoolsgroup.com/imap/) contains list of IMAP servers for most commonly used email providers, which can be used in case of any other email service provider.

Also, if you're using a Gmail account and the above code raises an error indicating that the credentials are incorrect, make sure you allow less secure apps on your account.



In [ ]:
status, messages = imap.select("INBOX")
# number of top emails to fetch

# total number of emails
messages = int(messages[0])
N = messages 
# N can be equated to any number depending upon the top N emails you want to scrape.
# Here N=messages means we want to scrape through all emails

In [ ]:
jobs = []    # make a list which will store all the objects whose subject is matches our required string

for i in range(messages, messages-N, -1): # going from the top to the bottom, the newest email messages got the highest id number
    
    res, msg = imap.fetch(str(i), "(RFC822)")    #  fetches the email message by ID using the standard format specified in RFC 822
    
    for response in msg:
        
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            
            # decode the email subject human readable unicode
            subject = decode_header(msg["Subject"])[0][0]
            
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode()
                
            # email sender
            from_ = msg.get("From")
           
            if subject.lower() == "Thank you for applying".lower():
                jobs.append([from_, msg, subject])
            
    
print(jobs)

In [1]:
file = open('Jobs.csv', 'w+', newline ='')  # creating a file names jobs.csv

In [ ]:
# writing the list in a file 
with file:     
    write = csv.writer(file) 
    write.writerow(['Sender', 'Object', 'Subject'])
    write.writerows(jobs)

In [ ]:
# logging out from your account
imap.close()
imap.logout()

In [ ]:
# import the external file and obtain the filtered out emails in a dataframe
import pandas as pd
df = pd.read_csv("jobs.csv")
df